In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data/train.csv', encoding='Big5')
data = data.replace('NR', '0')
data = np.array(data)

In [3]:
hour = 9
feature_num = 18
day_per_month = 20
per_month_row = feature_num * day_per_month
total_month = int(len(data)/per_month_row) 

In [4]:
month_data = []
for i in range(total_month):
    l = data[i * per_month_row: i * per_month_row  + per_month_row]
    month_data.append(l)  
    
hour_data = []
for i in range(len(month_data)):
    tmp = []
    for j in range(len(month_data[i])):
        tmp.append(month_data[i][j])
    hour_data.append(tmp)

In [5]:
total = []
for i in range(len(hour_data)):
    df = pd.DataFrame(hour_data[i])
    row, col = df.shape
    tmp = None
    for j in range(day_per_month):
   
        per_day = df.iloc[j*feature_num:j*feature_num+feature_num,:]
        per_day = per_day.iloc[:,3:]
        
        if tmp is not None:
            tmp = pd.concat([tmp.reset_index(drop=True), per_day.reset_index(drop=True)], axis=1, ignore_index=True)
        else:
            tmp = pd.DataFrame(per_day)    
    total.append(tmp)

In [6]:
xss = []
yss = []
for i in range(len(total)):
    df = total[i]
    row, col = df.shape
    for j in range(col-hour-1):
        xs = df.iloc[:,j:j+hour]
        xs = xs.values.ravel()
        xss.append(xs)
        yss.append(float(df.iloc[9,j+hour+1]))
xss = pd.DataFrame(xss).values.astype(np.float)
xss = np.column_stack(([1] * len(xss) ,xss))
yss = np.array(yss)

In [ ]:
max_iter = 10 ** 5
epochs = 10 ** 3
lr = 1
w = np.zeros(feature_num * hour+1)
w_lr = np.ones(feature_num * hour+1)

for t in range(epochs): 
    predict_arr = []
    for m in range(max_iter):
        predict = np.dot(xss,w)
        w_grad = (2 * np.dot(xss.T,(predict - yss)))
        predict_arr += [x**2 for x in (predict - yss)]
        w_lr = w_lr + w_grad ** 2
        w = w - lr/np.sqrt(w_lr) * w_grad

    print(np.sqrt(np.mean(predict_arr)))

17.915534868751127
8.148454361393531
8.143139733570573
8.139695726593676
8.137193636302907


In [147]:
test = pd.read_csv('data/test.csv', encoding='Big5', header=None)
test = test.replace('NR', '0')

In [183]:
ans = pd.read_csv('data/sampleSubmission.csv', encoding='Big5')
test_feature = 18
total_test = []
row, col = test.shape
test_number = int(row/test_feature)
print(test_number)
for i in range(test_number):
    df = test.iloc[i*test_feature: (i+1)*test_feature, 2:]
    xs = df.values.ravel().astype(np.float)
    xs = np.concatenate(([1], xs))
    val = np.dot(xs,w)
    ans.iloc[i,1] = val
# ans.to_csv('data/0223.csv',index=False)

240


In [193]:
# import pickle
# with open('model/0223.pkl', 'wb') as f:
#     pickle.dump(w, f)
# with open('model/0223.pkl', 'rb') as f:
#       mynewlist = pickle.load(f)